In [1]:
import os, sys
from pathlib import Path
import cortex
import nibabel as nb
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as pl
import time
import platform
import pickle
import pandas as pd
from matplotlib.colors import Normalize
from prf_expect.utils import io

In [2]:
subject = "sub-002"

In [3]:
cwd = os.getcwd()
figure_result_dir = Path(cwd).parent / "figures"
settings = io.load_settings()
data_dir = Path(settings['general']['data_dir'], "data") / "derivatives" / "prf_data" / subject / "ses-1"

Loading settings from /Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/pRF_expect_pub/pRF_expect_analysis/prf_expect/settings.yml


In [4]:
print('Running on {}'.format(platform.node()))
print('cortex.database.default_filestore: {}'.format(cortex.database.default_filestore))
print('cortex.options.usercfg: {}'.format(cortex.options.usercfg))

Running on Smoldering-Corpse-Bar.local
cortex.database.default_filestore: /Users/dionysus/Documents/pycortex/db
cortex.options.usercfg: /Users/dionysus/Library/Application Support/pycortex/options.cfg


In [5]:
# polar angle and prf mapping stat maps
TRMI_0_fn = os.path.join(data_dir, "TRMI-type1-simulation-0", f"{subject}_ses-1_space-fsaverage_surprise-TRMI.npy")
TRMI_p015_fn = os.path.join(data_dir, "TRMI-type1-simulation-p015", f"{subject}_ses-1_space-fsaverage_surprise-TRMI.npy")
TRMI_p03_fn = os.path.join(data_dir, "TRMI-type1-simulation-p03", f"{subject}_ses-1_space-fsaverage_surprise-TRMI.npy")
TRMI_n015_fn = os.path.join(data_dir, "TRMI-type1-simulation-n015", f"{subject}_ses-1_space-fsaverage_surprise-TRMI.npy")
TRMI_n03_fn = os.path.join(data_dir, "TRMI-type1-simulation-n03", f"{subject}_ses-1_space-fsaverage_surprise-TRMI.npy")
TRMI_0 = np.load(TRMI_0_fn, allow_pickle=True)
TRMI_p015 = np.load(TRMI_p015_fn, allow_pickle=True)
TRMI_p03 = np.load(TRMI_p03_fn, allow_pickle=True)
TRMI_n015 = np.load(TRMI_n015_fn, allow_pickle=True)
TRMI_n03 = np.load(TRMI_n03_fn, allow_pickle=True)
TRMI_0[np.isnan(TRMI_0)] = 0
TRMI_p015[np.isnan(TRMI_p015)] = 0
TRMI_p03[np.isnan(TRMI_p03)] = 0
TRMI_n015[np.isnan(TRMI_n015)] = 0
TRMI_n03[np.isnan(TRMI_n03)] = 0

In [ ]:
static_imgs = False
web_view = True

In [7]:
cx_TRMI_n015 = cortex.Vertex(TRMI_n015.T, subject="fsaverage", cmap="BrBG_r", vmin=-0.3, vmax=0.3)
cx_TRMI_n03 = cortex.Vertex(TRMI_n03.T, subject="fsaverage", cmap="BrBG_r", vmin=-0.3, vmax=0.3)
cx_TRMI_0 = cortex.Vertex(TRMI_0.T, subject="fsaverage", cmap="BrBG_r", vmin=-0.3, vmax=0.3)
cx_TRMI_p015 = cortex.Vertex(TRMI_p015.T, subject="fsaverage", cmap="BrBG_r", vmin=-0.3, vmax=0.3)
cx_TRMI_p03 = cortex.Vertex(TRMI_p03.T, subject="fsaverage", cmap="BrBG_r", vmin=-0.3, vmax=0.3)

if web_view:
    print("creating web view")
    # webview for movies and comparisons
    for d, nm, depth in zip([cx_TRMI_n03.raw, cx_TRMI_n015, cx_TRMI_0.raw, cx_TRMI_p015.raw, cx_TRMI_p03.raw, ], 
                            ["TRMI_n03", "TRMI_n015", "TRMI_0", "TRMI_p015", "TRMI_p03"],
                            [0,0,0,0,0]):
        ds = cortex.Dataset(**{nm: d})
        handle = cortex.webgl.show(data=ds, recache=True, labels_visible=(), overlays_visible=('rois',))
        file_pattern = "{base}_{view}_{nm}.png"
        time.sleep(20.0)
        # projection parameters
        basic = dict(
            radius=300, depth=depth, specularity=0, unfold=0.5, contrast=0
        )  # projection=['orthographic'],
        # different views available, more views can be added  and the
        # existing list can be removed
        views = dict(
            myview=dict(altitude=94, azimuth=189, pivot=0),
            # lateral=dict(altitude=90.5, azimuth=181, pivot=180),
            # medial=dict(altitude=90.5, azimuth=0, pivot=180),
            # front=dict(altitude=90.5, azimuth=0, pivot=0),
            # back=dict(altitude=90.5, azimuth=181, pivot=0),
            # top=dict(altitude=0, azimuth=180, pivot=0),
            # bottom=dict(altitude=180, azimuth=0, pivot=0),
        )
        # utility functions to set the different views
        prefix = dict(
            altitude="camera.",
            azimuth="camera.",
            pivot="surface.{subject}.",
            radius="camera.",
            unfold="surface.{subject}.",
            depth="surface.{subject}.",
            specularity="surface.{subject}.",
            contrast="surface.curvature.",
        )
        _tolists = lambda p: {prefix[k] + k: [v] for k, v in p.items()}
        _combine = lambda a, b: (lambda c: [c, c.update(b)][0])(dict(a))
        # Save images by iterating over the different views and surfaces
        for view, vparams in views.items():
            # Combine basic, view, and surface parameters
            params = _combine(basic, vparams)
            # Set the view
            handle._set_view(**_tolists(params))
            # Save image
            filename = file_pattern.format(base=subject, view=view, nm=nm)

            output_path = os.path.join(
                figure_result_dir, filename
            )
            handle.getImage(output_path, size=(3840, 2160))
            # the block below trims the edges of the image:
            # wait for image to be written
            while not os.path.exists(output_path):
                pass
            time.sleep(1.5)
            # try:
            #     import subprocess
            #     subprocess.call(['convert', '-trim', output_path, output_path])
            # except:
            #     pass
        # Close the window!
        handle.close()

creating web view
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 42935
Unknown parameter surface.curvature.contrast!
Stopping server
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 10384
Unknown parameter surface.curvature.contrast!
Stopping server
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 11652
Unknown parameter surface.curvature.contrast!
Stopping server
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 32283
Unknown parameter surface.curvature.contrast!
Stopping server
Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 50388
Unknown parameter surface.curvature.contrast!
Stopping server
